In [3]:
#Check python's version
from platform import python_version
print("The current python's version is: ",python_version())

#Check the path of the current Python interpreter used to execute the script.
import sys
print("The path of the current Python interpreter: ",sys.executable)

The current python's version is:  3.9.17
The path of the current Python interpreter:  /Users/laura/opt/anaconda3/envs/Tesina39/bin/python


In [4]:
#Check for library's version 
import pkg_resources

library_name = 'warnings'
try:
    library_version = pkg_resources.get_distribution(library_name).version
    print(f"The version of {library_name} in use is: {library_version}")
except pkg_resources.DistributionNotFound:
    print(f"{library_name} ist's install in the project.")

warnings ist's install in the project.


<h1 style="background-color:#eaeaea;font-family:Calibri;color:#fe0b50;font-size:200%;text-align:center;border-radius: 50px;padding: 10px">From Json e XML to df, cleansing and from df to Database</h1>

- [Libraries & Utilities](#1)

- Point A - Retrive data + Point B - Store Data into a structure
    - [JSON](#2-1)
    - [XML](#2-2)
- Point C - Transform data 
    - [Cleaning](#3-1)
    - [Tokenizzation/Splitting](#3-2)
    - [Normalizzation (nlp library)](#3-3)
- Point D - Final Files and Mongo Db  
    - [Create a new files](#4-1)
    - [Store data in Mongo](#4-2)

<a id = '1'></a>
<h1 style="font-size: 150%;">Libraries & Utilities</h1>

In [5]:
#Install all the libraries the project needs
#pip install -r requirements.txt 

import os
import json #Point A_JSON
import xml.etree.ElementTree as ET #Point A_XML
import pandas as pd #Point B
import nltk #Point C Normalizzation
nltk.download('punkt') #Point C Normalizzation
from nltk.tokenize import word_tokenize #Point C Normalizzation
import pymongo #Point D2_Mongo

import warnings #only for os
warnings.filterwarnings("ignore") #only for os

[nltk_data] Downloading package punkt to /Users/laura/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Libraries:
1. ***os***
-> Used to interact with the operating system.
2. ***json***
-> Used to parse and read JSON type files and manipulate their tree structure.
3. ***xml.etree.ElementTree***
-> Used to parse and read JSON type files and manipulate their tree structure.
4. ***pandas as pd***
-> Used to parse data; among its main functionalities is the creation of the data structure called DataFrame.
5. ***nltk***
-> Used for natural language processing tasks such as tokenization, stemming, tagging, parsing, and semantic reasoning.
6. ***nltk.tokenize***
-> Part of NLTK, it provides various tokenization methods for breaking text into individual words or sentences.
7. ***pymongo***
-> Python driver for MongoDB. It allows Python applications to interact with MongoDB databases, enabling operations such as inserting, updating, querying, and deleting data.
8. ***warnings***
-> Used to handle warnings during code execution.

<a id = '2-1'></a>
<h1 style="font-size: 150%;">Point A - Retrive data + Point B - Store Data into a structure - JSON </h1>

In [6]:
#Point A
#Retrives data JSON

name_json='Input/jsondata.json'

#DEF that reads a JSON file and transforms it into a df by tagging columns
def jsonfile(name_json):
    #Reads and transforms into a dictionary
    with open(name_json) as f:
        data = json.load(f)

    #Point B 
    #Stores Data into a structure 
    #pd.json_normalize converts the JSON into a DataFrame
    df = pd.json_normalize(data)

    return df

#Calls the jsonfile function and prints the result
df_j = jsonfile(name_json)
#print(df_j.head(3))

#Deletes empty coloumns 
df_j.drop(columns=['json'], inplace=True)
print(df_j.head(3))

#Export to Excel
df_j.to_excel('Dirty_JSON.xlsx', index=False) 
print("\n\nFile: Dirty_JSON.xlsx exported!")

   id first_name last_name                 email  gender      ip_address  \
0   1      Riley    Peddie      rpeddie0@ftc.gov    Male   25.165.93.196   
1   2   Gabriela    Godlee     ggodlee1@digg.com  Female  156.228.163.91   
2   3      Major   Fitchen  mfitchen2@rambler.ru    Male    203.95.8.234   

  only_digit only_char  
0     bnn6P4    dg1YLS  
1     k6h8Z1    yo204Z  
2     s8kSG8    q4lX1J  


File: Dirty_JSON.xlsx exported!


<a id = '2-2'></a>
<h1 style="font-size: 150%;">Point A - Retrive data + Point B - Store Data into a structure - XML </h1>

In [7]:
#Point A
#Retrives data XML

name_xml='Input/dataset.xml'

#DEF that reads a XML file and transforms it into a df by tagging columns
def xml(name_xml):
    tree = ET.parse(name_xml)
    #Reads xml file and returns the object of typ
    #(the hierarchical structure of the XML tree)
  
    root = tree.getroot()
    #Obtains the root node of the object tree
    #(the starting point for navigating within the XML structure)

    tag=[] #Empty array/list
    #Populates the array/list and deletes duplicate values 
    #(SET is dangerous because even if it cleans up duplicates, it messes up the order of the elements!!!)
    for elem in root.iter():
        if elem.tag not in tag: 
            tag.append(elem.tag)

    #Creates column header with tags and an empty df
    heading = list(tag)
    df = pd.DataFrame(columns=heading)

    rows_data = []
    #Iterates on XML elements and populates the DataFrame
    for elem in root.iter():
        data = {}
        for tag in heading:
            if elem.find(tag) is not None:
                data[tag] = elem.find(tag).text
            else:
                data[tag] = None
        #Adds the row data dictionary to the list of rows
        rows_data.append(data)

    #Point B 
    #Stores Data into a structure 
    #Creates a new DataFrame from the rows
    df = pd.DataFrame(rows_data)

    #Deletes empty rows from id coloumn
    df.dropna(subset=['id'], inplace=True)

    #Deletes empty coloumns 
    df.drop(columns=['dataset', 'json','record'], inplace=True)

    return df

#Calls the xml function and prints the result
df_x = xml(name_xml)
#sorted_dfxml = df_x.sort_values(by=['last_name'], ascending=False) #Sorted by last_name
print(df_x.head(3))

#Export to Excel
df_x.to_excel('Dirty_XML.xlsx', index=False) 
print("\n\nFile: Dirty_XML.xlsx exported!")

   id first_name last_name                    email   gender       ip_address  \
1   1       Nert  MacAskie  nmacaskie0@trellian.com  Agender  183.240.159.252   
17  2      Moise   Jobbins     mjobbins1@jalbum.net     Male    132.3.198.177   
27  3      Suzie  Lawlings      slawlings2@digg.com   Female     66.42.231.49   

   only_digit only_char  
1      jb5D80    2kpKME  
17     z30NIA    bp9HJF  
27     wrwPNM    1r0SWC  


File: Dirty_XML.xlsx exported!


<a id = '3-1'></a>
<h1 style="font-size: 150%;">Point C - Transform data - CLEANING</h1>

In [8]:
#Point C
#Transforms data - CLEANING (!@#$luca1984.!@# => luca)

#JSON
#DEF that does several cleaning 
dfc = df_j.copy()
ty = 'JSON'
def clng(dfc, ty):
    #Checks column type
    print(f'\nThe coloumns of df from Input {ty} are:\n',dfc.dtypes,'\n')

    #Converts all df's values to lower case, if the type's columns is object
    df_low = dfc.apply(lambda x: x.str.lower() if(x.dtype == 'object') else x)
    
    #Change '@' & '.' in '_'
    df_low['email'] = df_low['email'].str.replace(r'[@]', '_', regex=True)
    df_low['ip_address'] = df_low['ip_address'].str.replace(r'[.]', ' ', regex=True)

    #Deletes chars from 'only_digit' 
    df_low['only_digit'] = df_low['only_digit'].str.replace('[a-zA-Z]', '', regex=True)

    #Deletes numbers from 'only_char' 
    df_low['only_char'] = df_low['only_char'].str.replace('\d+', '', regex=True)

    print(f"\nDf {ty} after creansing:\n", (df_low.head(1)))

    return df_low

#Calls the DEF clng on JSON
df_jcln = clng(dfc,ty)

#XML
#Using the DEF clng on XML 
dfc = df_x.copy()
ty ='XML'
df_xcln = clng(dfc,ty)


The coloumns of df from Input JSON are:
 id             int64
first_name    object
last_name     object
email         object
gender        object
ip_address    object
only_digit    object
only_char     object
dtype: object 


Df JSON after creansing:
    id first_name last_name             email gender     ip_address only_digit  \
0   1      riley    peddie  rpeddie0_ftc.gov   male  25 165 93 196         64   

  only_char  
0     dgyls  

The coloumns of df from Input XML are:
 id            object
first_name    object
last_name     object
email         object
gender        object
ip_address    object
only_digit    object
only_char     object
dtype: object 


Df XML after creansing:
   id first_name last_name                    email   gender       ip_address  \
1  1       nert  macaskie  nmacaskie0_trellian.com  agender  183 240 159 252   

  only_digit only_char  
1        580     kpkme  


<a id = '3-2'></a>
<h1 style="font-size: 150%;">Point C - Transform data - TOKENIZATION/SPLITTING</h1>

In [9]:
#Point C
#Transforms data - TOKENIZATION/SPLITTING (xxx|yyy_zzz => xxx yyy zzz)

#JSON
#DEF that splitting email's values 
dfc = df_jcln.copy()
ty = 'JSON'
def splt(dfc, ty):
    #Creations of 3 new coloumns with 'mail' values 
    df_low = dfc.assign(email_name = dfc.email)
    df_low0 = df_low.assign(email_service = df_low.email)
    df_low1 = df_low0.assign(email_extens = df_low0.email)

    #Splitting email's value 
    df_low1['email_name'] = df_low1['email_name'].str.split('[-_\.]',expand = True,regex = True)[0]
    df_low1['email_service'] = df_low1['email_service'].str.split('[-_\.]',expand = True,regex = True)[1]
    df_low1['email_extens'] = df_low1['email_extens'].str.extract(r'[^_.]*(\..*)')

    #Checks duplicates
    dup_df = df_low1[df_low1.duplicated()]

    if dup_df.empty:
        print(f'\n\nNo duplicates {ty} ;) \n')
        print(f"Df {ty} after Tokenization/Splitting:\n", (df_low1.head(1)))
    else:
        print(dup_df)
        
    return df_low1

#Calls the DEF splt on JSON
df_jsplt = splt(dfc, ty)

#XML
#Using the DEF splt XML
dfc = df_xcln.copy()
ty = 'XML'
df_xsplt = splt(dfc, ty)



No duplicates JSON ;) 

Df JSON after Tokenization/Splitting:
    id first_name last_name             email gender     ip_address only_digit  \
0   1      riley    peddie  rpeddie0_ftc.gov   male  25 165 93 196         64   

  only_char email_name email_service email_extens  
0     dgyls   rpeddie0           ftc         .gov  


No duplicates XML ;) 

Df XML after Tokenization/Splitting:
   id first_name last_name                    email   gender       ip_address  \
1  1       nert  macaskie  nmacaskie0_trellian.com  agender  183 240 159 252   

  only_digit only_char  email_name email_service email_extens  
1        580     kpkme  nmacaskie0      trellian         .com  


<a id = '3-3'></a>
<h1 style="font-size: 150%;">Point C - Transform data - NORMALIZATION (nlp library)</h1>

In [10]:
#Che fare? Estrarre dati da colonna email? nome e estensione?

#Point C
#Transforms data - NORMALIZATION (nlp library)
#pip install nltk

#Apply() applies tokenization to the cells in the 'email' column. 
#Lambda takes the values from the 'email' column (x) and applies the word_tokenize() function to tokenize the tokens (words). 
#The tokens are saved in the new column 'email_tokens' in the df.

#JSON
#DEF that applies the normalizzation to email's
dfc = df_jsplt.copy()
ty = 'JSON'
old_df = df_j
def norm(dfc, ty, old_df):
    dfc['email_tokens'] = old_df['email'].apply(lambda x: word_tokenize(x)) #Take the first df created for have the entire mail adress
    print(f"\nDf {ty} after Normalization:\n", (dfc.head(3)))

    return dfc

#Calls the DEF norm on JSON
df_jnorm = norm(dfc, ty, old_df)

#XML
#Using the DEF norm on XML
dfc = df_xsplt.copy()
ty = 'XML'
old_df = df_x

df_xnorm = norm(dfc, ty, old_df)


Df JSON after Normalization:
    id first_name last_name                 email  gender      ip_address  \
0   1      riley    peddie      rpeddie0_ftc.gov    male   25 165 93 196   
1   2   gabriela    godlee     ggodlee1_digg.com  female  156 228 163 91   
2   3      major   fitchen  mfitchen2_rambler.ru    male    203 95 8 234   

  only_digit only_char email_name email_service email_extens  \
0         64     dgyls   rpeddie0           ftc         .gov   
1        681       yoz   ggodlee1          digg         .com   
2         88      qlxj  mfitchen2       rambler          .ru   

                 email_tokens  
0      [rpeddie0, @, ftc.gov]  
1     [ggodlee1, @, digg.com]  
2  [mfitchen2, @, rambler.ru]  

Df XML after Normalization:
    id first_name last_name                    email   gender       ip_address  \
1   1       nert  macaskie  nmacaskie0_trellian.com  agender  183 240 159 252   
17  2      moise   jobbins     mjobbins1_jalbum.net     male    132 3 198 177   
27  3 

<a id = '4-1'></a>
<h1 style="font-size: 150%;">Point D - Create a new File</h1>

In [11]:
#Point D1
#Creates a New file

df_jnorm.to_excel('Final_df_JSON.xlsx', index=False) 
df_xnorm.to_excel('Final_df_XML.xlsx', index=False) 

print("\n'Final_df_JSON.xlsx' and 'Final_df_XML.xlsx' exported!")

#Chose if you want delete 'Dirty' files
response = input("\nDo you want to delete Dirty_JSON.xlsx e Dirty_XML.xlsx ? (y/n): ")
response_lower = response.lower() #Conver in lower case

if response_lower == 'y':
    try:
        os.remove('Dirty_JSON.xlsx')
        os.remove('Dirty_XML.xlsx')
        print("\n\nDone! File Dirty_JSON.xlsx and e Dirty_XML deleted\n")
    except FileNotFoundError:
        print("\nHey! There isn't any 'dirty' files!'-_- \n")
else:  
        print("\n\nOk, 'dirty' files not deleted.\n")


'Final_df_JSON.xlsx' and 'Final_df_XML.xlsx' exported!


Done! File Dirty_JSON.xlsx and e Dirty_XML deleted



<a id = '4-2'></a>
<h1 style="font-size: 150%;">Point D - Store data in Mongo</h1>

In [12]:
#Point D2
#Store data into DB MONGO
#https://stackoverflow.com/questions/20167194/insert-a-pandas-dataframe-into-mongodb-using-pymongo

#pip install pymongo

#DEF that stores JSON df in Mongo DB
db_name = "Database_Json"
db_collection = "df_jexp"
dfc = df_jnorm.copy()
ty = 'JSON'

def mongo (db_name, db_collection, dfc, ty):
    myclient1 = pymongo.MongoClient("mongodb://localhost:27017/")
    mydb1 = myclient1[db_name]
    mycol1 = mydb1[db_collection]

    df = dfc
    fields = df.to_dict('records')
    mycol1.insert_many(fields) 

    print(f"\nDataframe {ty} exported to MongoDB!")

#Calls the DEF mongo on JSON
mongo_j = mongo (db_name, db_collection, dfc, ty)

#XML
#Using the DEF mongo on XML
db_name = "Database_XML"
db_collection = "df_xexp"
dfc = df_xnorm.copy()
ty = 'XML'

mongo_x = mongo (db_name, db_collection, dfc, ty)


Dataframe JSON exported to MongoDB!

Dataframe XML exported to MongoDB!
